# Loss Functions to be Explored

$$
\begin{aligned}
\mathcal{L}_\text{Weighted Cross-Entropy} &= - \sum_i \left[w_+ b_i \log p_i + w_- (1 - b_i) \log (1 - p_i)\right] = - w_+ \mathbf{b} \cdot \log \mathbf{p} + w_- \bar{\mathbf{b}} \cdot \log \bar{\mathbf{p}}\\

\mathcal{L}_\text{Dice} &= 1 - \frac{2\mathbf{p} \cdot \mathbf {b}}{\sum_i p_i + \sum_i b_i}\\

\mathcal{L}_\text{Focal-Tversky} &= \left( 1 - \frac{2\mathbf{p}\cdot\mathbf{b}}{\mathbf{p}\cdot\mathbf{b} + \beta \bar{\mathbf{p}} \cdot \mathbf{b} + (1 - \beta) \mathbf{p} \cdot \bar{\mathbf{b}}} \right)^\gamma.
\end{aligned}
$$

where $\mathbf{p}$ are the probabilities (softmaxxed outputs along channel dimension) from the model, and $\mathbf{b}$ are the binary mask (0, 1) values. We use $\bar{\cdot}$ to represent the complement, i.e. $\bar{x_i} = 1 - x_i$ in index notation.

There are actually many variations of the Dice loss, with a couple listed below,
$$
\mathcal{L}_\text{Dice} =
\begin{cases}
    &1 - \frac{2\mathbf{p} \cdot \mathbf {b}}{\sum_i p_i + \sum_i b_i},\\
    \\
    &1 - \frac{2\mathbf{p} \cdot \mathbf {b}}{\lvert p \rvert _2^2 + \lvert b \rvert _2 ^2}
\end{cases}
$$

# Model Selection

Loss Function: $\mathcal{L}_\text{Weighted Cross-Entropy} + \alpha \mathcal{L}_\text{Focal-Tversky}$,
and model parameters as listed: $\alpha = 1,\; \beta = 0.7,\; \gamma = 0.75,\; (w_-, w_+) = (0.1, 0.9)$.

Only tackling 2D mode, since FCN-ResNets can only handle 2D data.

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data

- FCN-ResNet50 had peak Val IoU of ~55% requiring ~20 epochs.
- FCN-ResNet101 had peak Val IoU of ~63% requiring ~60 epochs.
- U-Net had peak Val IoU of ~70% requiring 5 epochs.

Selected U-Net.

# Loss Selection

## $\mathcal{L}_\text{Weighted Cross-Entropy} + \alpha \mathcal{L}_\text{Focal-Tversky}$

$\alpha = 1,\; \beta = 0.7,\; \gamma = 0.75,\; (w_-, w_+) = (0.1, 0.9)$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- Unstable training and exploding gradients. This was not an issue during model selection and I do not believe I made any changes that should have impacted training stability. Nevertheless, we look to tackle issues as they arise.
- Trained well before running into NaNs. 2D U-Net had peak Val IoU of ~50% and 3D U-Net had peak Val IoU of ~79.3%.
Note that segmentations looked near perfect on evaluation, with 2D U-Net only doing badly on Carotid.

## $\mathcal{L}_\text{Weighted Cross-Entropy}$

$(w_-, w_+) = (0.1, 0.9)$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- No exploding gradients
- 2D U-Net 60% Val IoU in 2 epochs
- 3D U-Net 76% Val IoU in 5 epochs

$(w_-, w_+) = (0.2, 0.8)$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- 
- 2D U-Net 61% Val IoU in 1 epoch
- 3D U-Net 79% Val IoU in 5 epochs

$(w_-, w_+) = (0.3, 0.7)$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- No exploding gradients, and better performance than (0.1, 0.9)
- 2D U-Net 59% Val IoU in 3 epochs
- 3D U-Net 81% Val IoU in 7 epochs

## $\mathcal{L}_\text{Dice}$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- No exploding gradients
- 2D U-Net 66% Val IoU in 3 epochs
- 3D U-Net 82% Val IoU in 5 epochs

## $\mathcal{L}_\text{Focal-Tversky}$
$\beta = 0.7,\; \gamma = 0.75$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- Experienced exploding gradients for both 2D and 3D cases.
- 2D U-Net 49% Val IoU in 1 epoch
- 3D U-Net 79% Val IoU in 1 epoch
- I suspect exploding gradients to be attributed to the Focal-Tversky Loss function.

## $\mathcal{L}_\text{Weighted Cross-Entropy} + \alpha \mathcal{L}_\text{Dice}$

$\alpha = 1., (w_-, w_+) = (0.1, 0.9)$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- No exploding gradients
- 2D U-Net 63% Val IoU in 8 epochs
- 3D U-Net 80% Val IoU in 8 epochs

$\alpha = 1., (w_-, w_+) = (0.3, 0.7)$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- No exploding gradients
- 2D U-Net 65% Val IoU in 1 epoch
- 3D U-Net 81% Val IoU in 3 epochs

$\alpha = 1.5, (w_-, w_+) = (0.1, 0.9)$

- Training set: 1000 pieces of synthetic data
- Validation set: 3 pieces of real data
- No exploding gradients
- 2D U-Net
- 3D U-Net 83% Val IoU in 10 epochs

Selected DICE Loss

# Dataset Trials

## Test 1 -- Reducing training set size
- Training set: 100 pieces of synthetic data
- Validation set: 3 pieces of real data
- 2D U-Net 68% Val IoU in 29 epochs
- 3D U-Net 82% Val IoU in 28 epochs
- While the epoch count has shot up dramatically, it is important to note that times have greatly decreased, from ~30 mins to ~9mins on the same set-up, i.e. a 3x speed increase.

## Test 2 -- Padding validation set with synthetic data + 20% validation set data split
- Training set: 100 pieces of synthetic data
- Validation set: 3 pieces of real data + 20 pieces of synthetic data
- 2D U-Net 84% Val IoU in 33 epochs
- 3D U-Net 90% Val IoU in 42 epochs
- Note that Val IoU is no longer a super good metric if there is synthetic data in the validation pool.
- Observing evals on real data looks good. 2D U-Net performs badly on carotid, not picking up one of the vessels.
